In [ ]:
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import torch

In [ ]:
from model import diffusion_model
from grid import volume_grid
from MultiTask import mtl_network_swing, mtl_training_swing

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)

In [ ]:
f0 = 20.0
ref_date = datetime(2022, 9, 30)         
contract_start = datetime(2022, 10, 1)
contract_end = datetime(2022, 10, 31)
n_ex_dates = (contract_end - contract_start).days + 1
ex_dates = np.array([contract_start + timedelta(days = i) for i in range(n_ex_dates)])

In [ ]:
dimension = 1
alpha = 4.0 * torch.ones(dimension, device = device) # 3.0
sigma = 0.7 * torch.ones(dimension, device = device) # 0.25
hjm_model = diffusion_model.One_Factor_Hjm_Ln(f0, alpha, sigma, ref_date, ex_dates)

#rho = 0.3
#corr_mat = rho * torch.ones((dimension, dimension), device = device)
#corr_mat = corr_mat.fill_diagonal_(1.0)
#hjm_model = Multi_Factor_Hjm_Ln(f0, alpha, sigma, corr_mat, ref_date, ex_dates)

In [ ]:
Q_min = 20.0
Q_max = 25.0
q_min = 0.0 * torch.ones(len(ex_dates))
q_max = 1.0 * torch.ones(len(ex_dates))
grid = volume_grid.Volume_Grid(Q_min, Q_max, q_min, q_max, ex_dates)
strike_price = 20.0

In [ ]:
n_features = dimension + 1 # X + cash flow
q = 50 # 20
hidden_shared_layers = [q + n_features for i in range(2)]
Q_ints, Q_ints_for_training = grid.discretization_cum_vol()

In [ ]:
norm_Q = grid.Q_max - grid.Q_min
batch_size = 2048
mtl_nn_model = mtl_network_swing.Build_Mtl_Architecture_For_Swing(n_features, hidden_shared_layers, Q_ints_for_training, grid.Q_min, norm_Q, batch_size, q)

### Strat 1: EMA GradNorm

In [ ]:
optimizer_configs = {'n_iterations' : 200, 'n_batches' : 1, 'batch_size' : batch_size, 'lr_1': 1e-1, 'lr_2' : 1e-2, 'b' : 1.8, 'beta' : 0.7, 'tmp' : 1.0}

In [ ]:
my_mtl, loss_1 = mtl_training_swing.mtl_training_1(mtl_nn_model, hjm_model, optimizer_configs, grid, strike_price, Q_ints, 
                                                   Q_ints_for_training, False)

In [ ]:
n_pack, pack_size = 4, 500000
    
for i in range(1, n_ex_dates):
    my_mtl[i].sample_size = pack_size

In [ ]:
mtl_training_swing.valuation(my_mtl, hjm_model, grid, strike_price, n_pack, pack_size, Q_ints, Q_ints_for_training)

### Strat 2: basic weights strategy (equal or uniform weights)

In [ ]:
optimizer_configs = {'n_iterations' : 200, 'n_batches' : 1, 'batch_size' : batch_size, 'lr': 1e-1, 'weights_strat' : "equal"}

In [ ]:
mtl_nn_model = mtl_network_swing.Build_Mtl_Architecture_For_Swing(n_features, hidden_shared_layers, Q_ints_for_training, grid.Q_min, norm_Q, batch_size, q)

In [ ]:
my_mtl, loss_2 = mtl_training_swing.mtl_training_2(mtl_nn_model, hjm_model, optimizer_configs, grid, strike_price, Q_ints, Q_ints_for_training, False)

In [ ]:
for i in range(1, n_ex_dates):
    my_mtl[i].sample_size = pack_size

In [ ]:
mtl_training_swing.valuation(my_mtl, hjm_model, grid, strike_price, n_pack, pack_size, Q_ints, Q_ints_for_training)

###

In [ ]:
optimizer_configs = {'n_iterations' : 200, 'n_batches' : 1, 'batch_size' : batch_size, 'lr': 1e-1, 'weights_strat' : "uniform"}

In [ ]:
mtl_nn_model = Build_Mtl_Architecture_For_Swing(n_features, hidden_shared_layers, Q_ints_for_training, grid.Q_min, norm_Q, batch_size, q)

In [ ]:
my_mtl, loss_3 = mtl_training_swing.mtl_training_2(mtl_nn_model, hjm_model, optimizer_configs, grid, strike_price, Q_ints, Q_ints_for_training, False)

In [ ]:
for i in range(1, n_ex_dates):
    my_mtl[i].sample_size = pack_size

In [ ]:
mtl_training_swing.valuation(my_mtl, hjm_model, grid, strike_price, n_pack, pack_size, Q_ints, Q_ints_for_training)